In [43]:
import numpy as np
from numpy.random import binomial
import quantecon as qe
from numba import jit

In [33]:
def C(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        if x[t]==0:
            x[t+1] = binomial(1,.1) # if 0 (Low) move to 1 (High) with P=.1
        else:
            x[t+1] = binomial(1,.8)
    return(x)

In [41]:
qe.util.tic()
MC = C(0,100000)
time1 =qe.util.toc()

TOC: Elapsed: 0:00:0.25


In [42]:
np.mean(MC)

0.33152668473315267

In [45]:
C_numba = jit(C)

In [49]:
qe.util.tic()
C_numba(0,100000)
time2 = qe.util.toc()

TOC: Elapsed: 0:00:0.00


In [50]:
time1/time2

126.49421861962094

Or we could  have simply used the @jit decorator instruction. Now onto the Cython version:

In [51]:
%load_ext Cython

In [70]:
%%cython

import numpy as np
from numpy.random import binomial
import quantecon as qe
from numpy cimport float_t

def C_ython(double x0, int n):
    cdef int t
    
    x_np = np.zeros(n+1, dtype=int)
    b_np_0 = np.random.binomial(1,.1, n)
    b_np_1 = np.random.binomial(1,.8, n)
    
    cdef int_t [:] x = x_np
    cdef int_t [:] b_0 = b_np_0
    cdef int_t [:] b_1 = b_np_1
    
    x[0] = x0
    
    for t in range(1,n):
        if x[t]==0:
            x[t+1] = b_0[t] # if 0 (Low) move to 1 (High) with P=.1
        else:
            x[t+1] = b_1[t]
    return np.asarray(x)


Error compiling Cython file:
------------------------------------------------------------
...
    
    x_np = np.zeros(n+1, dtype=int)
    b_np_0 = np.random.binomial(1,.1, n)
    b_np_1 = np.random.binomial(1,.8, n)
    
    cdef int_t [:] x = x_np
        ^
------------------------------------------------------------

C:\Users\Glarange\.ipython\cython\_cython_magic_628b80209da3b4c808191ea7345c930b.pyx:14:9: 'int_t' is not a type identifier

Error compiling Cython file:
------------------------------------------------------------
...
    x_np = np.zeros(n+1, dtype=int)
    b_np_0 = np.random.binomial(1,.1, n)
    b_np_1 = np.random.binomial(1,.8, n)
    
    cdef int_t [:] x = x_np
    cdef int_t [:] b_0 = b_np_0
        ^
------------------------------------------------------------

C:\Users\Glarange\.ipython\cython\_cython_magic_628b80209da3b4c808191ea7345c930b.pyx:15:9: 'int_t' is not a type identifier

Error compiling Cython file:
-------------------------------------------------

TypeError: object of type 'NoneType' has no len()